<h1>Central Ohio</h1>
The National Restaurant Association estimates that the State of Ohio has approximately 22,547 eating and drinking places that earned $24.2 billion in 2018, with approximately 585,000 Ohioans, nearly 10% of all employed people in the State of Ohio, at work in the food service industry. Central Ohio, otherwise known as the Columbus metropolitan area, Ohio, is home to approximately 2,078,725 residents, and is considered the 32nd most populous area in the United States. Clearly, the the people of Ohio enjoy the food service industry, and the Columbus metropolitan area is a prime location for a new restaurant.

<h1>The Business Problem</h1>
This data science project will seek to provide demographics of the restaurant industry in central Ohio. This report is ideal for stakeholders who are intending to, or considering, opening a restaurant in Central Ohio. This report will provide valuable information regarding potential competitors, viable areas for your restaurant, and will provide an accurate landscape of the restaurant industry in Central Ohio. However, Central Ohio contains a large metropolitan area, with numerous suburban areas. Therefore, we will seek to provide the stakeholder with the most optimal area to open a restaurant.

<h1>The Data</h1>
This data science project will primarily use Foursquare API as a data source. Specifically, we will be gathering information on the restaurants, restaurant types, and attractions, within each neighborhood/zip code in the Columbus, OH area. Additionally, we have acquired population and income demographics for the area, from the US Census Bureau.

In [1]:
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
from geopy.geocoders import Nominatim  
import geocoder 
import requests
import lxml.html as lh
import pandas as pd
print("Libraries imported.")
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import csv
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.stats as stats

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda3

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py37h03978a9_0         3.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    tbb-2018.0.5               |       he980bc4_0         150 KB
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m
  tbb                pkgs/main/win-64::tbb-2018.0.5-he980bc4_0

The following packages will be UPDATED:

  conda               pkgs/main::conda-4.9.2-py37haa95532_0 --> conda-forge::conda-4.10.1-py37h03978a9_0




tbb-2018.0.5         | 150 KB    |        


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0




geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 KB     | ####7      |  47% 
geographiclib-1.50   | 34 KB     | ########## | 100% 
geographiclib-1.50   | 34 KB     | ########## | 100% 

geopy-2.1.0          | 64 KB    

In [3]:
#Loading and screening the data
url = 'https://raw.githubusercontent.com/zsmit15/Columbus_Restaurant_Industry/main/Columbus%20Zip%20Codes.csv'
df = pd.read_csv(url, index_col=0)
mean_age=df['Median Age'].mean()
mean_income=df['Median Income'].mean()
df['Median Age'].fillna(value=mean_age, inplace=True)
df['Median Income'].fillna(value=mean_income, inplace=True)
df.columns.map(type)
df.head()

,Neighborhood,Lat,Long,City,Population,Pop Density,Median Income,Median Age
Index,,,,,,,,
1,43001,40.08865,-82.61366,Alexandria,2570,37.4,76885.000000,46.9
2,43002,40.06333,-83.17390,Amlin,4303,1659.6,64425.000000,31.6
3,43003,40.41034,-82.97538,Ashley,2922,33.3,58155.000000,41.3
4,43004,40.01581,-82.80012,Blacklick,27227,812.5,92437.000000,34.4
5,43005,40.28618,-82.27706,Bladensburg,177,82.1,76448.232558,58.3


In [4]:
address = 'Columbus, OH'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Columbus are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Columbus are 39.9622601, -83.0007065.


In [5]:
#Creating a map of Columbus, using Latitude and Longitude values 
map_cbus = folium.Map(location=[latitude, longitude], zoom_start=8)

# Adding markers for zip codes to map
for lat, lng, city in zip(df['Lat'], df['Long'], df['City']):
    #label = '{}, {}'.format(city)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cbus)

<h1>The Methodology</h1>
This data science project will utilize the K-Means Cluster methodology, in order to cluster similar neighborhoods/zip codes in Columbus, OH, according to their similar composition regarding restaurants, attractions, age, and income. Each cluster will then be examined, and we will attempt to identify neighborhoods within each cluster, that lack the median number of restaurants for a specific restaurant type. The lack of a specific type of restaurant indicates a potential lack of supply, and given the comparison of similar restaurants in the cluster, there may be demand for that type of restaurant within the neighborhood.

In [6]:
CLIENT_ID = 'CTE5J53TDST50KXUQBBGUKDXLIBR4CEPMYTWQJH02NX5Q2I5' # your Foursquare ID
CLIENT_SECRET = 'LNLVR34ER2FJGPRH0JXITII5SSRNGC0V1G2PZRSS3EI3AIBT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CTE5J53TDST50KXUQBBGUKDXLIBR4CEPMYTWQJH02NX5Q2I5
CLIENT_SECRET:LNLVR34ER2FJGPRH0JXITII5SSRNGC0V1G2PZRSS3EI3AIBT


In [7]:
#Defining function, getting nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    empty_results = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)     
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        # Venues of 'Moore Park, Summerhill East' have recently become empty
        if len(results) != 0:
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        else:
            empty_results.append(name)
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("\n")
    for name in empty_results:
        print("The {} neighborhood doesn't contain venues.".format(name))
    
    return(nearby_venues)

In [12]:
#Creating dataframe for Columbus Venues
cbus_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Lat'],
                                   longitudes=df['Long'])

43001
43002
43003
43004
43005
43006
43008
43009
43010
43011
43013
43014
43015
43016
43017
43019
43021
43022
43023
43025
43026
43028
43029
43030
43031
43032
43033
43035
43036
43037
43040
43044
43045
43046
43050
43054
43055
43056
43060
43061
43062
43064
43065
43066
43067
43068
43070
43071
43072
43074
43076
43077
43078
43080
43081
43082
43084
43085
43101
43102
43103
43105
43106
43107
43109
43110
43111
43112
43113
43115
43116
43117
43119
43123
43125
43126
43127
43128
43130
43135
43136
43137
43138
43140
43142
43143
43144
43145
43146
43147
43148
43149
43150
43151
43152
43153
43154
43155
43156
43157
43158
43160
43162
43164
43201
43202
43203
43204
43205
43206
43207
43209
43210
43211
43212
43213
43214
43215
43217
43219
43220
43221
43222
43223
43224
43227
43228
43229
43230
43231
43232
43235
43240
43302


The 43003 neighborhood doesn't contain venues.
The 43006 neighborhood doesn't contain venues.
The 43009 neighborhood doesn't contain venues.
The 43014 neighborhood doesn't contain venues.
The 43

In [13]:
cbus_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,43001,40.08865,-82.61366,Ragamuffins Coffee,40.088862,-82.612611,Café
1,43001,40.08865,-82.61366,The Village Spa,40.089203,-82.612343,Spa
2,43001,40.08865,-82.61366,Parker's Park,40.088177,-82.615517,Park
3,43001,40.08865,-82.61366,Drummies Pizzeria,40.089859,-82.611763,Pizza Place
4,43002,40.06333,-83.17390,The Goat,40.061959,-83.173445,Bar


In [14]:
cbus_venues.groupby('Neighborhood')
cbus_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,43001,40.08865,-82.61366,Ragamuffins Coffee,40.088862,-82.612611,Café
1,43001,40.08865,-82.61366,The Village Spa,40.089203,-82.612343,Spa
2,43001,40.08865,-82.61366,Parker's Park,40.088177,-82.615517,Park
3,43001,40.08865,-82.61366,Drummies Pizzeria,40.089859,-82.611763,Pizza Place
4,43002,40.06333,-83.17390,The Goat,40.061959,-83.173445,Bar


In [15]:
# one hot encoding
cbus_onehot = pd.get_dummies(cbus_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cbus_onehot['Neighborhood'] = cbus_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cbus_onehot.columns[-1]] + list(cbus_onehot.columns[:-1])
cbus_onehot = cbus_onehot[fixed_columns]

cbus_onehot

,Neighborhood,ATM,Airport,Airport Service,American Restaurant,Argentinian Restaurant,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,...,Sushi Restaurant,Tourist Information Center,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,43001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,43001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,43001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,43001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,43002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,43002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,43002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,43002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,43004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
cbus_onehot.shape

(389, 131)

In [17]:
cbus_grouped = cbus_onehot.groupby('Neighborhood').mean().reset_index()
cbus_grouped

,Neighborhood,ATM,Airport,Airport Service,American Restaurant,Argentinian Restaurant,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,...,Sushi Restaurant,Tourist Information Center,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar
0,43001,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,43002,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,43004,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,43005,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,43008,0.000000,0.00,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,43010,0.000000,0.00,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,43011,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,43013,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,43015,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,43016,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
num_top_venues = 5
for hood in cbus_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = cbus_grouped[cbus_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

         venue  freq
0          Spa  0.25
1  Pizza Place  0.25
2         Park  0.25
3         Café  0.25
4  Music Store  0.00


                  venue  freq
0                   Gym   0.2
1          Home Service   0.2
2        Discount Store   0.2
3  Gym / Fitness Center   0.2
4                   Bar   0.2


          venue  freq
0           Gym  0.33
1           Pub  0.33
2    Hobby Shop  0.33
3     Pet Store  0.00
4  Noodle House  0.00


         venue  freq
0  Pizza Place   1.0
1          ATM   0.0
2  Post Office   0.0
3        Plaza   0.0
4   Playground   0.0


                 venue  freq
0  American Restaurant   0.2
1      Harbor / Marina   0.1
2            Pet Store   0.1
3              Brewery   0.1
4       Discount Store   0.1


                 venue  freq
0          Post Office   0.5
1  American Restaurant   0.5
2             Pharmacy   0.0
3         Noodle House   0.0
4    Outdoor Sculpture   0.0


               venue  freq
0  Health Food Store   1.0
1                ATM  

                 venue  freq
0           Food Court  0.25
1       Rental Service  0.25
2           Restaurant  0.25
3   Chinese Restaurant  0.25
4  Peruvian Restaurant  0.00


                      venue  freq
0                       Bar  0.13
1  Mediterranean Restaurant  0.07
2               Coffee Shop  0.07
3       American Restaurant  0.07
4        Salon / Barbershop  0.07


                venue  freq
0      Sandwich Place  0.09
1                Bank  0.09
2      Ice Cream Shop  0.04
3         Gaming Cafe  0.04
4  Chinese Restaurant  0.04


              venue  freq
0              Park  0.09
1  Business Service  0.09
2     Deli / Bodega  0.09
3         Rock Club  0.09
4        Restaurant  0.04


               venue  freq
0    Airport Service  0.50
1            Airport  0.25
2               Café  0.25
3           Pharmacy  0.00
4  Outdoor Sculpture  0.00


                  venue  freq
0          Home Service  0.25
1                  Park  0.25
2  Gym / Fitness Center  0.25
3     

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cbus_grouped['Neighborhood']

for ind in np.arange(cbus_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cbus_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,43001,Park,Pizza Place,Spa,Café,Wine Bar
1,43002,Home Service,Gym / Fitness Center,Bar,Gym,Discount Store
2,43004,Hobby Shop,Pub,Gym,Discount Store,Fast Food Restaurant
3,43005,Pizza Place,Wine Bar,Concert Hall,Convenience Store,Cosmetics Shop
4,43008,American Restaurant,Pizza Place,Pet Store,Sandwich Place,Fried Chicken Joint


<h1>Clustered Neighborhoods</h1>
Now we will cluster similar neighborhoods into five different clusters, using the K-Means Cluster methodology. Neighborhoods will be clustered, given their similar composition with population, population density, business demographics, an attractions.

In [20]:
#Comparing DataFrames, preparing for clustering
cbus_grouped_clustering = cbus_grouped.drop('Neighborhood', 1)
demographics = df[['Median Age', 'Median Income', 'Population', 'Pop Density']]
cbus_grouped_clustering = cbus_grouped_clustering.join(demographics)
mean_age = cbus_grouped_clustering['Median Age'].mean()
mean_income = cbus_grouped_clustering['Median Income'].mean()
mean_pop = cbus_grouped_clustering['Population'].mean()
mean_popdens = cbus_grouped_clustering['Pop Density'].mean()

cbus_grouped_clustering['Median Age'].fillna(value=mean_age, inplace=True)
cbus_grouped_clustering['Median Income'].fillna(value=mean_income, inplace=True)
cbus_grouped_clustering['Population'].fillna(value=mean_pop, inplace=True)
cbus_grouped_clustering['Pop Density'].fillna(value=mean_popdens, inplace=True)

print("cbus_grouped_clustering shape:", cbus_grouped_clustering.shape)
print("df shape:", neighborhoods_venues_sorted.shape)

cbus_grouped_clustering shape: (70, 134)
df shape: (70, 6)


In [21]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cbus_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 3, 4, 1, 3, 3, 4, 1])

In [22]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
cbus_merged = df
cbus_merged = cbus_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cbus_merged

,Neighborhood,Lat,Long,City,Population,Pop Density,Median Income,Median Age,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Index,,,,,,,,,,,,,,
1,43001,40.08865,-82.61366,Alexandria,2570,37.4,76885.000000,46.900000,1.0,Park,Pizza Place,Spa,Café,Wine Bar
2,43002,40.06333,-83.17390,Amlin,4303,1659.6,64425.000000,31.600000,1.0,Home Service,Gym / Fitness Center,Bar,Gym,Discount Store
3,43003,40.41034,-82.97538,Ashley,2922,33.3,58155.000000,41.300000,NaN,NaN,NaN,NaN,NaN,NaN
4,43004,40.01581,-82.80012,Blacklick,27227,812.5,92437.000000,34.400000,1.0,Hobby Shop,Pub,Gym,Discount Store,Fast Food Restaurant
5,43005,40.28618,-82.27706,Bladensburg,177,82.1,76448.232558,58.300000,3.0,Pizza Place,Wine Bar,Concert Hall,Convenience Store,Cosmetics Shop
6,43006,40.45874,-82.14575,Brinkhaven,537,8.7,54444.000000,45.900000,NaN,NaN,NaN,NaN,NaN,NaN
7,43008,39.93381,-82.48137,Buckeye Lake,2640,684.0,41033.000000,38.000000,4.0,American Restaurant,Pizza Place,Pet Store,Sandwich Place,Fried Chicken Joint
8,43009,40.17253,-83.64152,Cable,2614,29.3,91944.000000,41.700000,NaN,NaN,NaN,NaN,NaN,NaN
9,43010,39.99965,-83.62186,Catawba,242,404.9,65469.000000,37.500000,1.0,Post Office,American Restaurant,Discount Store,Fish & Chips Shop,Fast Food Restaurant


In [23]:
#Finding NaN values
cbus_merged.isnull().sum()

Neighborhood              0
Lat                       0
Long                      0
City                      0
Population                0
Pop Density               0
Median Income             0
Median Age                0
Cluster Labels           64
1st Most Common Venue    64
2nd Most Common Venue    64
3rd Most Common Venue    64
4th Most Common Venue    64
5th Most Common Venue    64
dtype: int64

In [24]:
#Dropping NaN values
cbus_merged.dropna(inplace=True)
cbus_merged['Cluster Labels'] = cbus_merged['Cluster Labels'].astype(int)
cbus_merged.isnull().sum()

Neighborhood             0
Lat                      0
Long                     0
City                     0
Population               0
Pop Density              0
Median Income            0
Median Age               0
Cluster Labels           0
1st Most Common Venue    0
2nd Most Common Venue    0
3rd Most Common Venue    0
4th Most Common Venue    0
5th Most Common Venue    0
dtype: int64

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, city, cluster in zip(cbus_merged['Lat'], cbus_merged['Long'], cbus_merged['City'], cbus_merged['Cluster Labels']):
    label = folium.Popup(str(city) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
        
       
map_clusters

<h1>Neighborhood Selection</h1>
Now we will select the most optimal neighborhood of zip codes to open a food service venue in the Columbus metropolitan area. Each neighborhood will be scored, using several variables, with the highest scored neighborhood being selected. The first variable will be mean income of the neighborhood, as this represents buying power for consumers. The second variable will be population density, as this should indicate the level of traffic to the restaurant and potential customer base. The third variable will be the number of restaurants within each category of the most common venue columns, divided by the number of zip codes, as this represents a strong restaurant industry within the neighborhood. All variables will be standardized with a Z-score, with the sum of the score being assigned to each neighborhood.

<h1>Red Neighborhood</h1>

In [26]:
red_cbus = cbus_merged.loc[cbus_merged["Cluster Labels"] == 0]
red_cbus

,Neighborhood,Lat,Long,City,Population,Pop Density,Median Income,Median Age,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Index,,,,,,,,,,,,,,
25,43031,40.16102,-82.66624,Johnstown,13737,59.6,90839.0,40.3,0,Flower Shop,Wine Bar,Concert Hall,Convenience Store,Cosmetics Shop
46,43068,39.95635,-82.78506,Reynoldsburg,54804,1093.6,80074.0,35.8,0,Gas Station,Discount Store,Wine Bar,Food & Drink Shop,Convenience Store
71,43116,39.77156,-83.06145,Commercial Point,1863,765.3,100338.0,32.7,0,Pizza Place,Post Office,Bar,Wine Bar,Dive Bar
92,43149,39.53834,-82.57212,Rockbridge,2293,16.2,80592.0,53.0,0,Construction & Landscaping,Trail,Wine Bar,Dive Bar,Fish & Chips Shop
107,43203,39.97310,-82.96898,Columbus,8014,2092.9,33202.0,33.0,0,Park,Bank,Plaza,Cajun / Creole Restaurant,Sandwich Place
118,43215,39.96635,-83.01282,Columbus,15291,1154.9,113359.0,30.7,0,Park,Business Service,Deli / Bodega,Rock Club,Gym
131,43232,39.92216,-82.87022,Columbus,47235,1486.6,47082.0,34.1,0,Home Service,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega


In [27]:
red_income = red_cbus['Median Income'].mean()
red_popdens = red_cbus['Pop Density'].mean()
red_venue = 66//33

print("The mean income is:", red_cbus['Median Income'].mean())
print("The mean age is:", red_cbus['Median Age'].mean())
print("The mean population is:", red_cbus['Population'].mean())
print("The mean population density is:", red_cbus['Pop Density'].mean())

The mean income is: 77926.57142857143
The mean age is: 37.08571428571429
The mean population is: 20462.428571428572
The mean population density is: 952.7285714285715


In [28]:
red_cbus.count()

Neighborhood             7
Lat                      7
Long                     7
City                     7
Population               7
Pop Density              7
Median Income            7
Median Age               7
Cluster Labels           7
1st Most Common Venue    7
2nd Most Common Venue    7
3rd Most Common Venue    7
4th Most Common Venue    7
5th Most Common Venue    7
dtype: int64

<h1>Red Neighborhood Summary</h1>
The red neighborhood, Cluster 0, consists of 33 zip codes. The mean income for the neighborhood is $81,071.95, with a mean age of 37 years old. The mean population of a zip code in the neighborhood is 21,271, with a population density of 988.39 per square mile. The neighborhood has 13 restaurants in the 1st most common venue category, 13 restaurants in the 2nd most common venue category, 15 restaurants in the 3rd most common venue category, 13 restaurants in the 4th most common venue category, and 12 restaurants in the 5th most common venue category.

<h1>Purple Neighborhood</h1>

In [29]:
purp_cbus = cbus_merged.loc[cbus_merged["Cluster Labels"] == 1]
purp_cbus

,Neighborhood,Lat,Long,City,Population,Pop Density,Median Income,Median Age,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Index,,,,,,,,,,,,,,
1,43001,40.08865,-82.61366,Alexandria,2570,37.4,76885.000000,46.900000,1,Park,Pizza Place,Spa,Café,Wine Bar
2,43002,40.06333,-83.17390,Amlin,4303,1659.6,64425.000000,31.600000,1,Home Service,Gym / Fitness Center,Bar,Gym,Discount Store
4,43004,40.01581,-82.80012,Blacklick,27227,812.5,92437.000000,34.400000,1,Hobby Shop,Pub,Gym,Discount Store,Fast Food Restaurant
9,43010,39.99965,-83.62186,Catawba,242,404.9,65469.000000,37.500000,1,Post Office,American Restaurant,Discount Store,Fish & Chips Shop,Fast Food Restaurant
14,43016,40.09861,-83.15376,Dublin,36886,786.2,116089.000000,38.000000,1,Chiropractor,Pizza Place,Pharmacy,Coffee Shop,Automotive Shop
19,43023,40.07526,-82.53181,Granville,14326,106.7,115583.000000,36.600000,1,Middle Eastern Restaurant,Soccer Field,Wine Bar,Dive Spot,Flower Shop
24,43030,39.96150,-82.41584,Jacksontown,108,453.9,76448.232558,39.196212,1,American Restaurant,Italian Restaurant,Bar,Wine Bar,Dive Spot
28,43035,40.18911,-82.99512,Lewis Center,27500,542.2,130603.000000,35.200000,1,Music Store,Café,Restaurant,Wine Bar,Dive Bar
36,43054,40.08085,-82.80342,New Albany,27441,529.0,143984.000000,35.500000,1,ATM,Spa,Gastropub,Pizza Place,Convenience Store


In [30]:
purp_income = purp_cbus['Median Income'].mean()
purp_popdens = purp_cbus['Pop Density'].mean()
purp_venue = 13//7

print("The mean income is:", purp_cbus['Median Income'].mean())
print("The mean age is:", purp_cbus['Median Age'].mean())
print("The mean population is:", purp_cbus['Population'].mean())
print("The mean population density is:", purp_cbus['Pop Density'].mean())

The mean income is: 80236.21956224351
The mean age is: 36.949888591800345
The mean population is: 21155.79411764706
The mean population density is: 1097.0382352941176


In [31]:
purp_cbus.count()

Neighborhood             34
Lat                      34
Long                     34
City                     34
Population               34
Pop Density              34
Median Income            34
Median Age               34
Cluster Labels           34
1st Most Common Venue    34
2nd Most Common Venue    34
3rd Most Common Venue    34
4th Most Common Venue    34
5th Most Common Venue    34
dtype: int64

<h1>Purple Neighborhood Summary</h1>
The purple neighborhood, Cluster 1, consists of 7 zip codes. The mean income for the neighborhood is $63,754.17, with a mean age of 40 years old. The mean population of a zip code in the neighborhood is 10,602, with a population density of 741.45 per square mile. The neighborhood has 3 restaurants in the 1st most common venue category, 0 restaurants in the 2nd most common venue category, 4 restaurants in the 3rd most common venue category, 1 restaurant in the 4th most common venue category, and 5 restaurants in the 5th most common venue category.

<h1>Blue Neighborhood</h1>

In [32]:
blue_cbus = cbus_merged.loc[cbus_merged["Cluster Labels"] == 2]
blue_cbus

,Neighborhood,Lat,Long,City,Population,Pop Density,Median Income,Median Age,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Index,,,,,,,,,,,,,,
26,43032,40.32822,-82.95917,Kilbourne,19,150.2,76448.232558,39.196212,2,Pizza Place,Park,Business Service,Wine Bar,Flower Shop
27,43033,39.96093,-82.60006,Kirkersville,300,260.3,58571.000000,57.400000,2,Post Office,Insurance Office,Bar,Wine Bar,Dive Bar
31,43040,40.26095,-83.35954,Marysville,35087,81.0,91928.000000,36.800000,2,Soccer Field,Pizza Place,Wine Bar,Convenience Store,Cosmetics Shop
65,43109,39.91257,-82.83370,Brice,77,190.0,53750.000000,47.900000,2,ATM,Argentinian Restaurant,Discount Store,Cosmetics Shop,Deli / Bodega
88,43145,39.56536,-83.25846,New Holland,2138,15.3,76065.000000,43.600000,2,Concert Hall,Music Venue,Wine Bar,Convenience Store,Cosmetics Shop
104,43164,39.58555,-83.11846,Williamsport,2371,16.2,57361.000000,36.000000,2,Post Office,Business Service,Deli / Bodega,Bar,Dive Spot
119,43217,39.81677,-82.93188,Columbus,2207,206.5,35854.000000,25.900000,2,Airport Service,Airport,Café,Wine Bar,Dive Spot


In [33]:
blue_income = blue_cbus['Median Income'].mean()
blue_popdens = blue_cbus['Pop Density'].mean()
blue_venue = 9//7

print("The mean income is:", blue_cbus['Median Income'].mean())
print("The mean age is:", blue_cbus['Median Age'].mean())
print("The mean population is:", blue_cbus['Population'].mean())
print("The mean population density is:", blue_cbus['Pop Density'].mean())

The mean income is: 64282.46179401993
The mean age is: 40.97088744588744
The mean population is: 6028.428571428572
The mean population density is: 131.35714285714286


In [34]:
blue_cbus.count()

Neighborhood             7
Lat                      7
Long                     7
City                     7
Population               7
Pop Density              7
Median Income            7
Median Age               7
Cluster Labels           7
1st Most Common Venue    7
2nd Most Common Venue    7
3rd Most Common Venue    7
4th Most Common Venue    7
5th Most Common Venue    7
dtype: int64

<h1>Blue Neighborhood Summary</h1>
The blue neighborhood, Cluster 2, consists of 7 zip codes. The mean income for the neighborhood is $73,417.42, with a mean age of 36 years old. The mean population of a zip code in the neighborhood is 24,966, with a population density of 939 per square mile. The neighborhood has 1 restaurants in the 1st most common venue category, 2 restaurants in the 2nd most common venue category, 2 restaurants in the 3rd most common venue category, 4 restaurants in the 4th most common venue category, and 0 restaurants in the 5th most common venue category.

<h1>Green Neighborhood</h1>

In [35]:
grn_cbus = cbus_merged.loc[cbus_merged["Cluster Labels"] == 3]
grn_cbus

,Neighborhood,Lat,Long,City,Population,Pop Density,Median Income,Median Age,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Index,,,,,,,,,,,,,,
5,43005,40.28618,-82.27706,Bladensburg,177,82.1,76448.232558,58.3,3,Pizza Place,Wine Bar,Concert Hall,Convenience Store,Cosmetics Shop
10,43011,40.31046,-82.68616,Centerburg,8018,37.5,89760.000000,39.0,3,Health Food Store,Wine Bar,Food & Drink Shop,Convenience Store,Cosmetics Shop
11,43013,40.23220,-82.68766,Croton,778,15.6,73583.000000,38.8,3,Stables,Golf Course,Wine Bar,Discount Store,Fast Food Restaurant
59,43101,39.46464,-82.74563,Adelphi,329,464.7,44643.000000,43.4,3,ATM,Pizza Place,American Restaurant,Bar,Dive Spot
67,43111,39.50207,-82.24238,Carbon Hill,291,279.4,81295.000000,40.8,3,Post Office,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop
68,43112,39.79671,-82.70358,Carroll,5357,59.3,105694.000000,44.7,3,Post Office,Construction & Landscaping,Pizza Place,Diner,Dive Bar
123,43222,39.96120,-83.03526,Columbus,4417,1215.6,31037.000000,35.9,3,Discount Store,Construction & Landscaping,Clothing Store,Park,Flower Shop
128,43229,40.08586,-82.97601,Columbus,54632,2393.3,52414.000000,33.1,3,Fast Food Restaurant,Sandwich Place,Shopping Mall,Grocery Store,Discount Store
130,43231,40.07927,-82.93894,Columbus,22393,2013.3,62042.000000,32.9,3,Golf Course,Pool,Dive Spot,Park,Discount Store


In [36]:
grn_income = grn_cbus['Median Income'].mean()
grn_popdens = grn_cbus['Pop Density'].mean()
grn_venue = 18//9

print("The mean income is:", grn_cbus['Median Income'].mean())
print("The mean age is:", grn_cbus['Median Age'].mean())
print("The mean population is:", grn_cbus['Population'].mean())
print("The mean population density is:", grn_cbus['Pop Density'].mean())

The mean income is: 68546.24806201551
The mean age is: 40.766666666666666
The mean population is: 10710.222222222223
The mean population density is: 728.9777777777778


In [37]:
grn_cbus.count()

Neighborhood             9
Lat                      9
Long                     9
City                     9
Population               9
Pop Density              9
Median Income            9
Median Age               9
Cluster Labels           9
1st Most Common Venue    9
2nd Most Common Venue    9
3rd Most Common Venue    9
4th Most Common Venue    9
5th Most Common Venue    9
dtype: int64

<h1>Green Neighborhood Summary</h1>
The green neighborhood, Cluster 3, consists of 9 zip codes. The mean income for the neighborhood is $70,662.13, with a mean age of 39 years old. The mean population of a zip code in the neighborhood is 10,884, with a population density of 782.96 per square mile. The neighborhood has 3 restaurants in the 1st most common venue category, 3 restaurants in the 2nd most common venue category, 3 restaurants in the 3rd most common venue category, 5 restaurants in the 4th most common venue category, and 4 restaurants in the 5th most common venue category.

In [38]:
#Creating DataFrame for Z-Scoring
scores = [['Red', red_income, red_popdens, red_venue], ['Purple', purp_income, purp_popdens, purp_venue], ['Blue', blue_income, blue_popdens, blue_venue], ['Green', grn_income, grn_popdens, grn_venue]]
columns = ['Neighborhood', 'Income Score', 'Density Score', 'Venue Score', 'Total']
neigh_scores = pd.DataFrame(scores).drop([0], axis=1)
neigh_scores

,1,2,3
0,77926.571429,952.728571,2
1,80236.219562,1097.038235,1
2,64282.461794,131.357143,1
3,68546.248062,728.977778,2


In [39]:
neigh_scores.apply(stats.zscore)

,1,2,3
0,0.789260,0.611413,1.0
1,1.141262,1.003205,-1.0
2,-1.290172,-1.618561,-1.0
3,-0.640349,0.003943,1.0


In [40]:
neigh_scores.apply(stats.zscore).sum(axis=1)

0    2.400672
1    1.144467
2   -3.908733
3    0.363594
dtype: float64

<h1>Score Table</h1>
Most Suitable Neighborhood
Therefore, the highest scoring neighborhood is the red neighborhood, otherwise known as Cluster 0. The red neighborhood had a higher mean income and mean population density relative to the other neighborhoods. Therefore, it is an ideal location for a new restaurant.

<h1>Zip Code Selection</h1>
Now that we have identified the most optimal group of zip codes for a new restaurant, we can move on to selecting the actual zip code. Our goal is to find zip codes that are under-supplied with restaurants, relative to other zip codes in the neighborhood. We will create a new DataFrame, and remove the variables we used for clustering. We will then collect venues from FourSquare for each of our zip codes. Then we will remove non-food service venues, and compile a list of restaurants for each zip code.

In [41]:
#Removing variables from the DataFrame
red_cbus.drop(['Population','Pop Density', 'Median Income', 'Median Age', 'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue'], axis=1,inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [42]:
#Getting nearby venues for the zip codes
zip_venues = getNearbyVenues(names=red_cbus['Neighborhood'],
                                   latitudes=df['Lat'],
                                   longitudes=df['Long'])

43031
43068
43116
43149
43203
43215
43232


The 43116 neighborhood doesn't contain venues.
The 43215 neighborhood doesn't contain venues.


In [43]:
#Grouping venues by zip code
zip_venues.groupby('Neighborhood')
zip_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,43031,40.08865,-82.61366,Ragamuffins Coffee,40.088862,-82.612611,Café
1,43031,40.08865,-82.61366,The Village Spa,40.089203,-82.612343,Spa
2,43031,40.08865,-82.61366,Parker's Park,40.088177,-82.615517,Park
3,43031,40.08865,-82.61366,Drummies Pizzeria,40.089859,-82.611763,Pizza Place
4,43068,40.06333,-83.17390,The Goat,40.061959,-83.173445,Bar


In [44]:
zip_onehot = pd.get_dummies(zip_venues[['Venue Category']], prefix="", prefix_sep="")
zip_onehot['Neighborhood'] = zip_venues['Neighborhood'] 
fixed_columns = [zip_onehot.columns[-1]] + list(zip_onehot.columns[:-1])
zip_onehot = zip_onehot[fixed_columns]
zip_onehot

,Neighborhood,American Restaurant,Bar,Brewery,Café,Chinese Restaurant,Discount Store,Fried Chicken Joint,Gym,Gym / Fitness Center,Harbor / Marina,Hobby Shop,Home Service,Park,Pet Store,Pizza Place,Pub,Sandwich Place,Spa
0,43031,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,43031,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,43031,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,43031,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,43068,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,43068,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,43068,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,43068,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,43068,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,43149,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [45]:
#Listing each type of venue within a zip code
red_grouped = zip_onehot.groupby('Neighborhood').mean().reset_index()
red_grouped

,Neighborhood,American Restaurant,Bar,Brewery,Café,Chinese Restaurant,Discount Store,Fried Chicken Joint,Gym,Gym / Fitness Center,Harbor / Marina,Hobby Shop,Home Service,Park,Pet Store,Pizza Place,Pub,Sandwich Place,Spa
0,43031,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.25,0.0,0.25,0.000000,0.0,0.25
1,43068,0.0,0.2,0.0,0.00,0.0,0.2,0.0,0.200000,0.2,0.0,0.000000,0.2,0.00,0.0,0.00,0.000000,0.0,0.00
2,43149,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.0,0.00,0.0,0.00,0.333333,0.0,0.00
3,43203,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,1.00,0.000000,0.0,0.00
4,43232,0.2,0.0,0.1,0.00,0.1,0.1,0.1,0.000000,0.0,0.1,0.000000,0.0,0.00,0.1,0.10,0.000000,0.1,0.00


In [46]:
#Removing non-food service venues from the DataFrame
red_rest = red_grouped.loc[:,('Neighborhood', 'American Restaurant', 'BBQ Joint', 'Bar', 'Brewery', 'Café', 'Chinese Restaurant', 'Coffee Shop', 'Fried Chicken Joint', 'Italian Restaurant', 'Middle Eastern Restaurant', 'Pizza Place', 'Restaurant', 'Sandwich Place', 'Vegetarian / Vegan Restaurant', 'Wine Bar')]
red_rest

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:868: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_lowerdim(tup)


,Neighborhood,American Restaurant,BBQ Joint,Bar,Brewery,Café,Chinese Restaurant,Coffee Shop,Fried Chicken Joint,Italian Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Wine Bar
0,43031,0.0,NaN,0.0,0.0,0.25,0.0,NaN,0.0,NaN,NaN,0.25,NaN,0.0,NaN,NaN
1,43068,0.0,NaN,0.2,0.0,0.00,0.0,NaN,0.0,NaN,NaN,0.00,NaN,0.0,NaN,NaN
2,43149,0.0,NaN,0.0,0.0,0.00,0.0,NaN,0.0,NaN,NaN,0.00,NaN,0.0,NaN,NaN
3,43203,0.0,NaN,0.0,0.0,0.00,0.0,NaN,0.0,NaN,NaN,1.00,NaN,0.0,NaN,NaN
4,43232,0.2,NaN,0.0,0.1,0.00,0.1,NaN,0.1,NaN,NaN,0.10,NaN,0.1,NaN,NaN


In [47]:
#Finding the sum of food service venues within each zip code, sorted by ascending
column_list=list(red_rest)
column_list.remove("Neighborhood")
red_rest["Sum"] = red_rest[column_list].sum(axis=1)
red_rest.sort_values('Sum', ascending=True)

,Neighborhood,American Restaurant,BBQ Joint,Bar,Brewery,Café,Chinese Restaurant,Coffee Shop,Fried Chicken Joint,Italian Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Wine Bar,Sum
2,43149,0.0,NaN,0.0,0.0,0.00,0.0,NaN,0.0,NaN,NaN,0.00,NaN,0.0,NaN,NaN,0.0
1,43068,0.0,NaN,0.2,0.0,0.00,0.0,NaN,0.0,NaN,NaN,0.00,NaN,0.0,NaN,NaN,0.2
0,43031,0.0,NaN,0.0,0.0,0.25,0.0,NaN,0.0,NaN,NaN,0.25,NaN,0.0,NaN,NaN,0.5
4,43232,0.2,NaN,0.0,0.1,0.00,0.1,NaN,0.1,NaN,NaN,0.10,NaN,0.1,NaN,NaN,0.7
3,43203,0.0,NaN,0.0,0.0,0.00,0.0,NaN,0.0,NaN,NaN,1.00,NaN,0.0,NaN,NaN,1.0


<h1>Results</h1>
The most optimal zip codes for a new restaurant are 43016, 43212, 43065, 43070, 43127, which represent the cities of Dublin, Grandview, Powell, Rosewood, and Haydenville. The selected zip codes lack food service venues, per FourSquare. While these areas most likely do have food service venues, they are most likely not well-known or common venues, relative to other zip codes in the neighborhood. However, these zip codes have comparative venues, a high median income, and a high population density, which makes them ideal for a new restaurant.

<h1>Discussion</h1>
Overall, this was a very beneficial project, and it was great to be able practicing with data from an API, and using K-Means Clustering. However, FourSquare lacked a significant volume of venues for areas within the dataset, which I felt detracted from the results of the project.

<h1>Conclusion</h1>
Thank you for reviewing this data science project, which explored the food service industry in Central Ohio in order to find the optimal place for a new restaurant. Overall, I am pleased with the results. I can speak to three of the chosen zip codes, which I've personally seen, and I can honestly say that they are developing areas in the community with promising opportunities.